# De la estructura matemática a la computación científica, IA y física

**Espacios vectoriales y subespacios en código**
**Idea guía:** *no todo son flechas en* $\mathbb{R}^3$.

### Resumen

Este cuaderno formaliza el concepto de **espacio vectorial** como estructura definida por axiomas y luego traduce esa estructura a **representaciones computacionales**. Se estudian **subespacios**, **span** (subespacio generado), **independencia lineal** y su **detección computacional** (exacta y numérica). Finalmente se conectan estos conceptos con espacios de **funciones**, **matrices** y **datos**, enfatizando aplicaciones a computación científica, aprendizaje automático y física.

### Palabras clave

Espacios vectoriales, subespacios, independencia lineal, rango, núcleo, SVD, tolerancia numérica, bases, proyecciones, PCA.

### Convención

* Campo (escalares): típicamente $\mathbb{R}$ o $\mathbb{C}$.
* Vectores: objetos sobre los cuales definimos suma y multiplicación por escalar.
* En cómputo: distinguimos entre **cálculo exacto** (simbólico) y **cálculo numérico** (punto flotante).


In [1]:
import numpy as np
import sympy as sp

np.set_printoptions(precision=4, suppress=True)
sp.init_printing(use_unicode=True)

def is_close(a, b, tol=1e-9):
    return np.linalg.norm(np.array(a, dtype=float) - np.array(b, dtype=float)) <= tol

def rank_svd(A, tol=1e-10):
    """Rango numérico por SVD: cuenta singular values > tol."""
    A = np.array(A, dtype=float)
    s = np.linalg.svd(A, compute_uv=False)
    return int(np.sum(s > tol)), s

def nullspace_svd(A, tol=1e-10):
    """Base (aprox) del núcleo de A usando SVD."""
    A = np.array(A, dtype=float)
    U, s, Vt = np.linalg.svd(A)
    mask = s <= tol
    if np.all(~mask):
        return np.zeros((A.shape[1], 0))
    return Vt.T[:, mask]

def solve_least_squares(A, b):
    """Resuelve min||Ax-b||2 (numérico)."""
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    x, residuals, rnk, s = np.linalg.lstsq(A, b, rcond=None)
    return x, residuals, rnk, s

print("OK: setup cargado")

OK: setup cargado


## 1. Espacio vectorial: definición abstracta (y por qué importa)

Un **espacio vectorial** $V$ sobre un campo $\mathbb{F}$ es un conjunto con dos operaciones:

* **Suma**: $+: V \times V \to V$
* **Producto por escalar**: $\cdot: \mathbb{F} \times V \to V$

que satisfacen axiomas (asociatividad, conmutatividad, neutro, inverso aditivo, distributividad, compatibilidad con el producto del campo, etc.).

### Punto filosófico-técnico

En $\mathbb{R}^n$ solemos dibujar vectores como flechas, pero **ser vector no significa ser flecha**:

* Un polinomio puede ser un vector.
* Una función puede ser un vector.
* Una señal discretizada puede ser un vector.
* Una imagen (matriz de pixeles) puede ser un vector.

Lo importante no es “cómo se ve” el objeto, sino que cumpla los axiomas con operaciones bien definidas.

### Pregunta computacional

¿Cómo representamos un espacio vectorial abstracto en Python sin confundirlo con `np.array`?

La idea es separar:

* **Estructura**: axiomas + operaciones
* **Representación**: cómo almacenamos el objeto

In [2]:
from dataclasses import dataclass
from typing import Callable, Any

@dataclass
class VectorSpace:
    """Un espacio vectorial como paquete de operaciones.
    No almacena todos los vectores: define cómo operar.
    """
    field_name: str
    add: Callable[[Any, Any], Any]
    smul: Callable[[Any, Any], Any]
    zero: Callable[[], Any]

    def check_axioms_sanity(self, samples, scalars):
        """Chequeos parciales (no prueba formal), útil para depurar."""
        z = self.zero()

        # neutro aditivo: v+0=v
        for v in samples:
            if self.add(v, z) != v or self.add(z, v) != v:
                return False, "Falla neutro aditivo"

        # distributividad: a(u+v)=au+av
        for a in scalars:
            for u in samples:
                for v in samples:
                    left = self.smul(a, self.add(u, v))
                    right = self.add(self.smul(a, u), self.smul(a, v))
                    if left != right:
                        return False, "Falla distributividad a(u+v)=au+av"

        return True, "Chequeos básicos OK"

def Rn(n: int) -> VectorSpace:
    """R^n con tuplas (más 'abstracto' que np.array)."""
    def add(u, v):
        return tuple(u[i] + v[i] for i in range(n))

    def smul(a, v):
        return tuple(a * v[i] for i in range(n))

    def zero():
        return tuple(0.0 for _ in range(n))

    return VectorSpace(field_name="R", add=add, smul=smul, zero=zero)

V3 = Rn(3)
ok, msg = V3.check_axioms_sanity(
    samples=[(1,2,3), (0,0,0), (-1,0,2)],
    scalars=[-2, 0, 3]
)
ok, msg

(True, 'Chequeos básicos OK')

## 2. Polinomios como vectores

Sea $P_d$ el espacio de polinomios reales de grado $\le d$.

Un polinomio $p(x)=a_0+a_1x+\dots+a_dx^d$ se representa como su vector de coeficientes:
$$
(a_0,a_1,\dots,a_d)\in\mathbb{R}^{d+1}.
$$

El objeto matemático es el polinomio; el arreglo es una codificación.
Construiremos `PolyVec` para mantener clara esa identidad.

In [3]:
from dataclasses import dataclass

@dataclass(frozen=True)
class PolyVec:
    coeffs: tuple  # (a0, a1, ..., ad)

    def degree(self):
        for i in range(len(self.coeffs)-1, -1, -1):
            if self.coeffs[i] != 0:
                return i
        return -1

    def __call__(self, x):
        s = 0
        for i, a in enumerate(self.coeffs):
            s += a * (x**i)
        return s

def Pd(d: int) -> VectorSpace:
    def add(p: PolyVec, q: PolyVec):
        return PolyVec(tuple(p.coeffs[i] + q.coeffs[i] for i in range(d+1)))

    def smul(a: float, p: PolyVec):
        return PolyVec(tuple(a * p.coeffs[i] for i in range(d+1)))

    def zero():
        return PolyVec(tuple(0.0 for _ in range(d+1)))

    return VectorSpace(field_name="R", add=add, smul=smul, zero=zero)

P3 = Pd(3)

p = PolyVec((1, -2, 0, 3))      # 1 - 2x + 0x^2 + 3x^3
q = PolyVec((0,  5, 1, 0.5))    # 5x + x^2 + 0.5x^3

r = P3.add(p, q)
s = P3.smul(2, p)

print("p(x)=", p.coeffs, "grado", p.degree())
print("q(x)=", q.coeffs, "grado", q.degree())
print("p+q =", r.coeffs)
print("2p  =", s.coeffs)
print("p(2)=", p(2))

p(x)= (1, -2, 0, 3) grado 3
q(x)= (0, 5, 1, 0.5) grado 3
p+q = (1, 3, 1, 3.5)
2p  = (2, -4, 0, 6)
p(2)= 21


## 3. Subespacios: definición y test computacional

Un subconjunto $W\subseteq V$ es un **subespacio** si:

1. $0 \in W$
2. Si $u,v\in W$ entonces $u+v\in W$
3. Si $u\in W$ y $a\in\mathbb{F}$ entonces $au\in W$

En práctica no enumeramos $W$. Lo describimos por:

* generadores (span)
* ecuaciones lineales (núcleo / soluciones $Ax=0$)

Esto reduce el problema a **rango**, **dependencia lineal** y **núcleo**.

## 4. Span, rango y pertenencia

Dado $S={v_1,\dots,v_k}\subset V$,

$$ \operatorname{span}(S) = \left\{ \sum_{i=1}^k a_i v_i \right\}. $$

En $\mathbb{R}^n$, si $A=[v_1\ v_2\ \dots\ v_k]$:

* Span = espacio columna de $A$
* $\dim(\text{span})=\operatorname{rank}(A)$
* Dependencia si $\operatorname{rank}(A)<k$

Para pertenencia $b\in \text{span}$: resolver $Ax=b$.
Con ruido: mínimos cuadrados + residuo.

In [4]:
v1 = np.array([1, 0, 1], dtype=float)
v2 = np.array([2, 0, 2], dtype=float)   # = 2*v1 (dependiente)
v3 = np.array([0, 1, 0], dtype=float)

A = np.column_stack([v1, v2, v3])
print("A=\n", A)

# Rango exacto (enteros)
A_sp = sp.Matrix(A.astype(int))
print("Rango exacto:", A_sp.rank())

# Rango numérico (SVD)
rank_num, svals = rank_svd(A, tol=1e-12)
print("Rango numérico:", rank_num)
print("Singular values:", svals)

A=
 [[1. 2. 0.]
 [0. 0. 1.]
 [1. 2. 0.]]
Rango exacto: 2
Rango numérico: 2
Singular values: [3.1623 1.     0.    ]


In [5]:
b_in = np.array([3, 5, 3], dtype=float)   # 3*v1 + 5*v3 (debería estar dentro)
b_out = np.array([1, 1, 0], dtype=float)  # probablemente fuera

for name, b in [("b_in", b_in), ("b_out", b_out)]:
    x, residuals, rnk, s = solve_least_squares(A, b)
    b_hat = A @ x
    resid = np.linalg.norm(b_hat - b)

    print("\n", name)
    print("x* =", x)
    print("Ax =", b_hat)
    print("||Ax-b|| =", resid)


 b_in
x* = [0.6 1.2 5. ]
Ax = [3. 5. 3.]
||Ax-b|| = 0.0

 b_out
x* = [0.1 0.2 1. ]
Ax = [0.5 1.  0.5]
||Ax-b|| = 0.7071067811865476


## 5. Independencia lineal y núcleo

$v_1,\dots,v_k$ son **linealmente independientes** si:
$$
\sum_{i=1}^k a_i v_i = 0 \Rightarrow a_1=\dots=a_k=0.
$$

Con $A=[v_1\ \dots\ v_k]$:

* Independientes ⇔ $\operatorname{rank}(A)=k$
* Dependientes ⇔ existe $x\neq 0$ tal que $Ax=0$

Es decir: **dependencia** ⇔ **núcleo no trivial**.

In [6]:
# Núcleo exacto (sympy)
null_exact = A_sp.nullspace()
print("Base del núcleo (exacta):", null_exact)

# Núcleo aproximado (SVD)
ns = nullspace_svd(A, tol=1e-12)
print("Base del núcleo (aprox, columnas):\n", ns)

# Dependencia esperada: 2*v1 - v2 = 0  ->  x=(2,-1,0)
x = np.array([2, -1, 0], dtype=float)
print("A @ x =", A @ x)

Base del núcleo (exacta): [Matrix([
[-2],
[ 1],
[ 0]])]
Base del núcleo (aprox, columnas):
 [[ 0.8944]
 [-0.4472]
 [ 0.    ]]
A @ x = [0. 0. 0.]


## 6. Extraer una base desde un conjunto generador

En práctica recibimos vectores redundantes y queremos una **base** del span.

Estrategia numérica simple:

1. iniciar con matriz vacía
2. ir agregando columnas
3. aceptar una columna si aumenta el rango (según tolerancia)

In [7]:
def basis_from_columns(vectors, tol=1e-10):
    """Selecciona subconjunto de vectores (columnas) que forman base del span.
    Agrega si aumenta el rango numérico.
    """
    selected = []
    A = np.zeros((len(vectors[0]), 0))
    current_rank = 0

    for v in vectors:
        A_candidate = np.column_stack([A, v])
        r, _ = rank_svd(A_candidate, tol=tol)
        if r > current_rank:
            selected.append(v)
            A = A_candidate
            current_rank = r

    return selected, A, current_rank

vectors = [v1, v2, v3]
basis, B, dim = basis_from_columns(vectors, tol=1e-12)

print("Dimensión del span:", dim)
print("Base (columnas):\n", B)

Dimensión del span: 2
Base (columnas):
 [[1. 0.]
 [0. 1.]
 [1. 0.]]


## 7. Espacios de funciones (y cómo existen en código)

El conjunto de funciones $f:\mathbb{R}\to\mathbb{R}$ con suma y producto por escalar es un espacio vectorial (infinito-dimensional).

En computador, dos enfoques:

* (A) coeficientes en una base (polinomios, Fourier)
* (B) muestreo (discretización): representar $f$ por $(f(x_1),\dots,f(x_n))\in\mathbb{R}^n$

Esto conecta directamente con física (métodos numéricos) e IA (señales, series temporales, imágenes).

In [8]:
xs = np.linspace(-1, 1, 200)

# 'Vectores-función' muestreados
f1 = np.ones_like(xs)   # 1
f2 = xs                 # x
f3 = xs**2              # x^2

F = np.column_stack([f1, f2, f3])

rankF, sF = rank_svd(F, tol=1e-10)
print("Rango (subespacio generado por {1,x,x^2} muestreados):", rankF)
print("Singular values:", sF[:6])

Rango (subespacio generado por {1,x,x^2} muestreados): 3
Singular values: [14.9887  8.2059  4.0181]


In [9]:
g = 3 - 2*xs + 0.5*xs**2          # polinomio (debe caer en el span)
h = np.sin(np.pi * xs)            # no cae exacto en el span, pero se proyecta

for name, y in [("g (polinomio)", g), ("h (seno)", h)]:
    coeffs, residuals, rnk, s = solve_least_squares(F, y)
    y_hat = F @ coeffs
    err = np.linalg.norm(y_hat - y) / np.linalg.norm(y)

    print("\n", name)
    print("coeficientes ~", coeffs)
    print("error relativo ||proj - y|| / ||y|| =", err)


 g (polinomio)
coeficientes ~ [ 3.  -2.   0.5]
error relativo ||proj - y|| / ||y|| = 1.47662428995453e-16

 h (seno)
coeficientes ~ [-0.      0.9406  0.    ]
error relativo ||proj - y|| / ||y|| = 0.633427828898048


## 8. Datos como subespacios: dependencia aproximada y tolerancia

En datos reales, una variable puede ser casi combinación lineal de otras (por ruido).
La dependencia deja de ser “sí/no” exacto y pasa a ser **numérica**.

La SVD detecta esto: si hay singular values muy pequeñas, el dataset vive cerca de un subespacio de baja dimensión.

In [10]:
rng = np.random.default_rng(7)
n = 200

x1 = rng.normal(size=n)
x2 = rng.normal(size=n)

noise = 1e-3 * rng.normal(size=n)
x3 = x1 + 2*x2 + noise   # casi dependiente

X = np.column_stack([x1, x2, x3])

r1, s1 = rank_svd(X, tol=1e-12)
r2, s2 = rank_svd(X, tol=1e-3)

print("Singular values:", s1)
print("Rango con tol=1e-12:", r1)
print("Rango con tol=1e-3 :", r2)

Singular values: [33.4873 12.5583  0.0054]
Rango con tol=1e-12: 3
Rango con tol=1e-3 : 3


## 9. Subespacio de mejor aproximación: SVD y proyección (puente a PCA)

Si (X) está centrada:
$$
X = U\Sigma V^T
$$
Las columnas de (V) son direcciones (bases) en el espacio de features.
Tomar las primeras $k$ define un subespacio de dimensión $k$ que captura la mayor “energía”.

In [11]:
Xc = X - X.mean(axis=0, keepdims=True)

U, s, Vt = np.linalg.svd(Xc, full_matrices=False)
energy = (s**2) / np.sum(s**2)
cum_energy = np.cumsum(energy)

print("Singular values:", s)
print("Energía por componente:", energy)
print("Energía acumulada:", cum_energy)

k = 2
Vk = Vt[:k, :].T    # d x k
Z = Xc @ Vk         # n x k
X_hat = Z @ Vk.T    # reconstrucción

recon_error = np.linalg.norm(Xc - X_hat) / np.linalg.norm(Xc)
print("Error relativo de reconstrucción con k=2:", recon_error)

Singular values: [33.1872 12.4908  0.0052]
Energía por componente: [0.8759 0.1241 0.    ]
Energía acumulada: [0.8759 1.     1.    ]
Error relativo de reconstrucción con k=2: 0.00014772090805974616


## 10. Subespacios como soluciones: núcleos y ecuaciones

Si $A\in\mathbb{R}^{m\times n}$, entonces:

$$
\mathcal{N}(A)={x\in\mathbb{R}^n: Ax=0}
$$

es un subespacio.

Ejemplo: subespacio de $\mathbb{R}^3$ ortogonal a $n=(1,2,3)$:

$$
W={x: n^T x=0}
$$

Es el núcleo de la matriz $A=[n^T]$.

In [12]:
nvec = sp.Matrix([[1, 2, 3]])  # 1x3
W_basis = nvec.nullspace()
print("Base exacta de W (x ortogonal a n):", W_basis)

for b in W_basis:
    print("Verificación n^T b =", (nvec * b)[0])

Base exacta de W (x ortogonal a n): [Matrix([
[-2],
[ 1],
[ 0]]), Matrix([
[-3],
[ 0],
[ 1]])]
Verificación n^T b = 0
Verificación n^T b = 0


## Síntesis conceptual

1. Espacio vectorial = estructura (axiomas), no un dibujo.
2. Computación = representaciones finitas (coeficientes, muestras, matrices).
3. Subespacios: por generadores (span) o por ecuaciones (núcleo).
4. Independencia ↔ rango; dependencia ↔ núcleo no trivial.
5. En datos reales: dependencia es aproximada → SVD + tolerancias.
6. SVD/PCA = subespacios + proyección + mejor aproximación.

## Ejercicios

### Ej1 — Span y pertenencia

Construye $A$ con columnas $v_1=(1,1,0)$, $v_2=(2,0,1)$, $v_3=(3,1,1)$.

1. Calcula el rango.
2. Decide si $b=(1,0,1)$ pertenece al span usando `lstsq` y residuo.

### Ej2 — Dependencia con ruido

Genera $x_3=x_1-4x_2+\epsilon$ con distintos niveles de ruido.
Estudia cómo cambia el rango numérico al variar la tolerancia.

### Ej3 — Funciones muestreadas

En $[-1,1]$: genera columnas $1,x,x^2,x^3$.
Proyecta (\cos(\pi x)) sobre el span y reporta error relativo.

### Ej4 — Núcleo exacto

Encuentra una base exacta del subespacio de $\mathbb{R}^4$ definido por:
$$
\begin{cases}
x_1+x_2+x_3+x_4=0\\
2x_1-x_2+0x_3+x_4=0
\end{cases}
$$
usando `sympy.nullspace()`.

In [13]:
# EJ1
v1 = np.array([1,1,0], float)
v2 = np.array([2,0,1], float)
v3 = np.array([3,1,1], float)
A = np.column_stack([v1, v2, v3])

print("A=\n", A)
print("rank:", rank_svd(A, tol=1e-12)[0])

b = np.array([1,0,1], float)
x, residuals, rnk, s = solve_least_squares(A, b)
print("x*:", x)
print("||Ax-b||:", np.linalg.norm(A@x - b))

# EJ4
Aeq = sp.Matrix([
    [1, 1, 1, 1],
    [2,-1, 0, 1],
])
print("\nNullspace exacta EJ4:")
print(Aeq.nullspace())

A=
 [[1. 2. 3.]
 [1. 0. 1.]
 [0. 1. 1.]]
rank: 2
x*: [-0.3333  0.5     0.1667]
||Ax-b||: 0.4082482904638631

Nullspace exacta EJ4:
[Matrix([
[-1/3],
[-2/3],
[   1],
[   0]]), Matrix([
[-2/3],
[-1/3],
[   0],
[   1]])]
